In [ ]:
import cv2

import treasureArea as ta
import webCamera as wc

class blockDetect:
    def __init__(self):
        self.treasureArea = ta.TreasureArea()
        self.webCamera = wc.WebCamera(0)
    
    def detect(self):
        None
    
    def __imageProcess(self, image):

    def __findPoints(self, image):

    def __findArea(self, point)
